# Object tracking & motion analysis

## فیلتر کردن رنگ

### فیلتر بر اساس Hue (فام)

- بازه رنگ Hue از ۰ تا ۱۸۰ است

- بازه‌های فیلتر رنگ:
    - قرمز: ۱۶۵ تا ۱۵
    - سبز: ۴۵ تا ۷۵
    - آبی: ۹۰ تا ۱۲۰

### فیلتر بر اساس اشباع (Saturation) و روشنایی (Value/Brightness)

- معمولاً بازه فیلتر را برای اشباع و روشنایی از ۵۰ تا ۲۵۵ قرار می‌دهیم تا رنگ‌های واقعی را پوشش دهد
- بازه ۰ تا ۶۰ در اشباع نزدیک به سفید است
- بازه ۰ تا ۶۰ در روشنایی نزدیک به سیاه است


## Filter by color

In [ ]:
import cv2
import numpy as np

# Initialize webcam
cap = cv2.VideoCapture(0)

# define range of PURPLE color in HSV
lower_purple = np.array([125,0,0])
upper_purple = np.array([175,255,255])

# loop until break statement is exectured
while True:
    
    # Read webcam image
    ret, frame = cap.read()
    
    # Convert image from RBG/BGR to HSV so we easily filter
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


    # Use inRange to capture only the values between lower & upper_blue
    mask = cv2.inRange(hsv_img, lower_blue, upper_blue)

    # Perform Bitwise AND on mask and our original frame
    res = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow('Original', frame)  
    cv2.imshow('mask', mask)
    cv2.imshow('Filtered Color Only', res)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

این کد برای فیلتر کردن یک رنگ خاص (در اینجا بنفش) از تصویر دریافتی وب‌کم استفاده می‌کند. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`cv2` و `numpy`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- محدوده رنگ بنفش در فضای رنگی HSV با دو آرایه `lower_purple` و `upper_purple` تعریف می‌شود.
- یک حلقه بی‌نهایت اجرا می‌شود تا زمانی که کلید Enter فشرده شود.
- در هر تکرار حلقه:
    - یک فریم از وب‌کم خوانده می‌شود.
    - تصویر از فضای رنگی BGR به HSV تبدیل می‌شود تا فیلتر کردن رنگ راحت‌تر انجام شود.
    - با استفاده از تابع `cv2.inRange`، ماسکی ساخته می‌شود که فقط پیکسل‌هایی که در محدوده رنگ بنفش قرار دارند را نگه می‌دارد.
    - با استفاده از عملگر بیت‌به‌بیت AND، فقط بخش‌هایی از تصویر که رنگ بنفش دارند نمایش داده می‌شوند.
    - سه پنجره نمایش داده می‌شود: تصویر اصلی، ماسک، و تصویر فیلتر شده.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

**نکته:** در کد نمونه، متغیرهای `lower_blue` و `upper_blue` به اشتباه استفاده شده‌اند و باید به `lower_purple` و `upper_purple` تغییر داده شوند.


## Background Subtraction

این یک تکنیک بسیار مفید در بینایی ماشین است که به ما اجازه می‌دهد اجسام متحرک (پیش‌زمینه) را از پس‌زمینه در یک جریان ویدیویی جدا کنیم.

این الگوریتم‌ها اساساً درباره فریم موجود در تصویر (جریان ویدیو) یاد می‌گیرند و می‌توانند به‌طور دقیق ماسک پیش‌زمینه را شناسایی کنند. نتیجه این کار، یک تقسیم‌بندی دودویی از تصویر است که نواحی اجسام غیرثابت را برجسته می‌کند.

در OpenCV چندین الگوریتم تفریق پس‌زمینه مخصوص تحلیل ویدیو وجود دارد:

- **BackgroundSubtractorMOG** – یک الگوریتم جداسازی پیش‌زمینه/پس‌زمینه مبتنی بر مخلوط گاوسی. هر پیکسل پس‌زمینه را با یک مخلوطی از توزیع‌های گاوسی مدل می‌کند و با یک تقریب آنلاین مدل را به‌روزرسانی می‌کند.
- **BackgroundSubtractorMOG2** – نسخه بهبود یافته MOG که سازگاری بهتری با صحنه‌های متغیر و سایه‌ها دارد.
- **BackgroundSubtractorGMG** – از تخمین آماری تصویر پس‌زمینه و تقسیم‌بندی بیزی پیکسل‌به‌پیکسل استفاده می‌کند.

این الگوریتم‌ها به طور گسترده در کاربردهایی مانند ردیابی اشیا، تشخیص حرکت و نظارت تصویری استفاده می‌شوند.


### 1. Gaussian Mixture-based Background/Foreground Segmentation Algorithm

In [ ]:
# OpenCV 2.4.13 only
import numpy as np
import cv2

cap = cv2.VideoCapture('walking.avi')

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG()

while True:
    
    ret, frame = cap.read()

    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در یک ویدیو با استفاده از الگوریتم مخلوط گاوسی (MOG) در OpenCV نسخه 2.4.13 نوشته شده است. توضیح مراحل به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- ویدیوی `'walking.avi'` با استفاده از `cv2.VideoCapture` بارگذاری می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از ویدیو خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، ویدیو آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در ویدیو کاربرد دارد.


### اگر بخواهیم این روش را روی ورودی وب‌کم خود اجرا کنیم چه؟

In [ ]:
# OpenCV 2.4.13 only
import numpy as np
import cv2

# Intialize Webcam
cap = cv2.VideoCapture(0)

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG()

while True:
    
    ret, frame = cap.read()

    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در تصویر دریافتی از وب‌کم با استفاده از الگوریتم مخلوط گاوسی (MOG) در OpenCV نسخه 2.4.13 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از وب‌کم خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در تصویر وب‌کم کاربرد دارد.


### بیایید مدل مخلوط گاوسی تطبیقی بهبود یافته را برای تفریق پس‌زمینه امتحان کنیم

In [ ]:
# OpenCV 2.4.13
import numpy as np
import cv2

cap = cv2.VideoCapture('walking.avi')

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()

    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در یک ویدیو با استفاده از الگوریتم مخلوط گاوسی تطبیقی (MOG2) در OpenCV نسخه 2.4.13 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- ویدیوی `'walking.avi'` با استفاده از `cv2.VideoCapture` بارگذاری می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG2` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از ویدیو خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، ویدیو آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در ویدیو کاربرد دارد و نسبت به نسخه قبلی (MOG) عملکرد بهتری در شرایط نوری متغیر و سایه‌ها دارد.


### اعمال آن بر روی جریان وب‌کم خود

In [ ]:
# OpenCV 2.4.13
import numpy as np
import cv2

# Intialize Webcam
cap = cv2.VideoCapture(0)

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    
    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در تصویر دریافتی از وب‌کم با استفاده از الگوریتم مخلوط گاوسی تطبیقی (MOG2) در OpenCV نسخه 2.4.13 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG2` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از وب‌کم خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در تصویر وب‌کم کاربرد دارد و نسبت به نسخه قبلی (MOG) عملکرد بهتری در شرایط نوری متغیر و سایه‌ها دارد.


## تفریق پیش‌زمینه چیست؟

In [ ]:
import cv2
import numpy as np

# Initalize webacam and store first frame
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

# Create a flaot numpy array with frame values
average = np.float32(frame)

while True:
    # Get webcam frmae
    ret, frame = cap.read()
    
    # 0.01 is the weight of image, play around to see how it changes
    cv2.accumulateWeighted(frame, average, 0.01)
    
    # Scales, calculates absolute values, and converts the result to 8-bit
    background = cv2.convertScaleAbs(average)

    cv2.imshow('Input', frame)
    cv2.imshow('Disapearing Background', background)
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cv2.destroyAllWindows()
cap.release()

این کد برای حذف تدریجی پس‌زمینه از تصویر دریافتی وب‌کم با استفاده از میانگین‌گیری وزنی (Background Averaging) نوشته شده است. توضیح مراحل به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`cv2` و `numpy`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود و اولین فریم خوانده می‌شود.
- یک آرایه‌ی شناور (`float32`) با مقادیر اولین فریم ساخته می‌شود تا به عنوان میانگین اولیه پس‌زمینه استفاده شود.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - در هر تکرار، یک فریم جدید از وب‌کم خوانده می‌شود.
    - با استفاده از تابع `cv2.accumulateWeighted`، میانگین وزنی بین فریم فعلی و میانگین قبلی محاسبه می‌شود. پارامتر ۰.۰۱ وزن فریم جدید را تعیین می‌کند (هرچه این عدد بزرگ‌تر باشد، پس‌زمینه سریع‌تر به‌روزرسانی می‌شود).
    - با استفاده از `cv2.convertScaleAbs`، میانگین به تصویر ۸ بیتی تبدیل می‌شود تا قابل نمایش باشد.
    - دو پنجره نمایش داده می‌شود: یکی تصویر ورودی و دیگری پس‌زمینه‌ی محو شونده.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، تمام پنجره‌ها بسته شده و وب‌کم آزاد می‌شود.

این روش برای حذف تدریجی اجسام متحرک و نمایش پس‌زمینه‌ی ثابت کاربرد دارد.


### Background Substraction KKN
#### OpenCV 3.X only!

In [ ]:
# OpenCV 3.1.0
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorKNN()

while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    cv2.imshow('frame',fgmask)
    
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در تصویر دریافتی از وب‌کم با استفاده از الگوریتم KNN (K-Nearest Neighbors) در OpenCV نسخه 3.1.0 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- یک کرنل بیضوی با اندازه ۳x۳ برای عملیات مورفولوژیکی ساخته می‌شود.
- شیء `createBackgroundSubtractorKNN` برای تفریق پس‌زمینه با الگوریتم KNN ساخته می‌شود.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از وب‌کم خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - با استفاده از عملیات مورفولوژیکی `MORPH_OPEN`، نویزهای کوچک از ماسک حذف می‌شوند تا تصویر تمیزتری به دست آید.
    - ماسک نهایی نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در تصویر وب‌کم کاربرد دارد و نسبت به روش‌های قبلی در برخی شرایط عملکرد بهتری دارد.
